# Visualizing Stack Overflow Data in Python

In this notebook, we visualize posts on Stack Overflow from September 2017. The data was compiled from searches on the [Stack Exchange Data Explorer](https://data.stackexchange.com/stackoverflow/query/new). Location information was added using the [Google Maps API](https://developers.google.com/maps/).

<div id="contents"></div>
## Table of Contents
1. [Load the Data](#load)
1. [Visualize Completeness](#completeness)
1. [Visualize Time](#time)
1. [Visualize Tags](#tags)
1. [Explore Text](#explore)
1. [Plot Place](#place)

<div id="load"></div>
## Load the Data
*[Table of Contents](#contents)*

In [ ]:
# A nice library for reading in csv data
import pandas as pd

# load the data
posts = pd.read_csv('SeptemberPosts.csv')

# let's see what columns we have
posts.columns

What are some example values for each column?

In [ ]:
posts.head()

## Columns
What do all these columns mean?
- PostId = the id in Stack Overflow's database of this post
- Score = the score given to the post by people voting up and down on it
- PostType = What type of post is this?
- CreationData = When was this post posted?
- Title = The text in the title of the post
- Body = The HTML in the body of the post
- UserId = The id of the user who posted in the Stack Overflow database
- Reputation = The reputaiton of the user who posted
- Location = The location the user put down as their home on their profile
- Tags = Tags which are associated with this post
- QuestionId = The question this post is linked to

Let us convert CreationDate to a datetime type

In [ ]:
posts['CreationDate'] = pd.to_datetime(posts['CreationDate'])

<div id="completeness"></div>
## Visualizing Completeness
*[Table of Contents](#contents)*

I wonder what values PostType can take on

In [ ]:
# A library which most visualization libraries in Python are built on.
# We will start by using it to make some plots with pandas
import matplotlib.pyplot as plt

pd.DataFrame(posts['PostType']).apply(pd.value_counts).plot(kind='bar', subplots=True)
plt.show()

Most posts are either questions or answers. These types of posts serve very different purposes, so let's seperate them out.

I'd also like to know how complete our data is, so let's look at which fields have null values for the answers and questions.

In [ ]:
questions = posts[posts['PostType'] == 'Question']
answers = posts[posts['PostType'] == 'Answer']

In [ ]:
# A library to visualize holes in a dataset
import missingno as msno

msno.matrix(questions)

In [ ]:
msno.matrix(answers)

<div id="time"></div>
## Visualizing Time
*[Table of Contents](#contents)*

How long did it take to get an answer in September?

In [ ]:
threads = answers.groupby('QuestionId')

def get_aggregate(field, agg_fun, name):
    output = threads[field].agg(agg_fun)
    return pd.DataFrame({'PostId':output.index.values, name:output.values})

# get the statistics for questions that were answered
first_reply = get_aggregate('CreationDate', min, 'EarliestReply')
questions_with_answers = pd.merge(first_reply, questions, how='inner', on=['PostId'])

# get the time it took to get an answer
gap_between_ask_and_reply = (questions_with_answers['EarliestReply']-questions_with_answers['CreationDate'])
# convert to minutes
gap_between_ask_and_reply /= pd.Timedelta(minutes=1)

# find the median
median = gap_between_ask_and_reply.median()
print('Median answer time for questions asked and answered in September is', median, 'min.')

# plot
gap_between_ask_and_reply.hist(bins = 100)
plt.yscale('log')
plt.ylabel('Number of Questions')
plt.xlabel('Time in Minutes')
plt.show()

<div id="tags"></div>
## Visualize Tags
*[Table of Contents](#contents)*

What do we ask about when we ask about women and men and programing?

In [ ]:
import wordcloud

def text_to_wordcloud(series, title):
    text = ' '.join(list(series))
    # make a wordcloud from the text
    title_wordcloud = wordcloud.WordCloud().generate(text)
    # we want the words in our cloud to all be the same color
    title_wordcloud.recolor(color_func=lambda word, **kwargs:'white')
    # turn the wordcloud into an image
    plt.imshow(title_wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(title + ' (' + str(len(series)) + ' questions)')
    plt.show()

def show_tags(words, title):
    tags = questions[questions['Body'].str.lower().str.contains('|'.join(words))]['Tags']
    text = tags.str.replace('<',' ').replace('>',' ')
    text_to_wordcloud(text, title)


show_tags([' he ', ' his ', ' him ', ' himself '], 'Male Tags')
show_tags([' she ', ' hers ', ' her ', ' herself '], 'Female Tags')

<div id="explore"></div>
## Text Visualization
*[Table of Contents](#contents)*

What did users put in the titles of their python posts?

In [ ]:
python_titles = questions[questions['Tags'].str.contains('python')]['Title'].str.lower()
text_to_wordcloud(python_titles, 'Python Titles')

Are there other ways to look at word frequencies?

In [ ]:
# lets us use graphviz in python
from pydotplus import graphviz
# to display the final Image
from IPython.display import Image
from collections import Counter
import re

node_counter = 0

class Node:
    def __init__(self, word, count, matching_strings, graph, reverse=False, branching=3, highlight=False):
        global node_counter
        if highlight:
            self.node = graphviz.Node(node_counter, label=word+'\n'+str(count), peripheries=2, fontsize=20)
        else:
            self.node = graphviz.Node(node_counter, label=word+'\n'+str(count))
        node_counter += 1
        graph.add_node(self.node)
        self.generate_children(matching_strings, graph, reverse, branching)
    
    def generate_children(self, matching_strings, graph, reverse, branching):
        if len(matching_strings) == 0:
            return
        matching_strings = matching_strings[matching_strings.apply(len) > 0]
        all_children = Counter(matching_strings.apply(lambda x:x[-1 if reverse else 0]))
        children = all_children.most_common(branching)
        for word, count in children:
            if not reverse:
                child_matches = matching_strings[matching_strings.apply(lambda x:x[0]) == word].apply(lambda x:x[1:])
                c_node = Node(word, count, child_matches, graph=graph, reverse=reverse, branching=branching)
                graph.add_edge(graphviz.Edge(self.node, c_node.node))
            else:
                child_matches = matching_strings[matching_strings.apply(lambda x:x[-1]) == word].apply(lambda x:x[:-1])
                c_node = Node(word, count, child_matches, graph=graph, reverse=reverse, branching=branching)
                graph.add_edge(graphviz.Edge(c_node.node, self.node))
        left_over = sum(all_children.values()) - sum([x[1] for x in children])
        if left_over > 0:
            c_node = Node('...', left_over, [], graph=graph, reverse=reverse, branching=branching)
            if reverse:
                graph.add_edge(graphviz.Edge(c_node.node, self.node))
            else:
                graph.add_edge(graphviz.Edge(self.node, c_node.node))

def build_tree(root_string, suffixes, prefixes):
    graph = graphviz.Dot()
    root = Node(root_string, len(suffixes), suffixes, graph, reverse=False, highlight=True)
    root.generate_children(prefixes, graph, True, 3)
    return Image(graph.create_png())

def get_end(string, sub_string, reverse):
    side = 0 if reverse else -1
    return [x for x in re.split(r'[^\w]+', string.lower().split(sub_string)[side]) if len(x) > 0]

def select_text(phrase):
    instances = questions[questions['Title'].str.lower().str.contains(phrase)]['Title']
    suffixes = instances.apply(lambda x: get_end(x, phrase, False))
    prefixes = instances.apply(lambda x: get_end(x, phrase, True))
    return build_tree(phrase, suffixes, prefixes)

select_text('file using python')

In [ ]:
select_text('tableau')

<div id="place"></div>
## Plot Places
*[Table of Contents](#contents)*